In [0]:
%run ./00_setup_and_config


In [0]:
# NOTEBOOK 07: FEATURE IMPORTANCE ANALYSIS (MULTI-MODEL)
# ============================================================================
# Databricks notebook source
# MAGIC %md
# MAGIC # Feature Importance Analysis - All Commodities and Model Versions
# MAGIC 
# MAGIC Analyzes which prediction features are most important for forecasting returns.
# MAGIC Runs for all configured commodities and model versions.

# COMMAND ----------

# MAGIC %run ./00_setup_and_config

# COMMAND ----------

import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns

# COMMAND ----------

# MAGIC %md
# MAGIC ## Feature Extraction and Analysis

# COMMAND ----------

def extract_features(predictions, current_price, eval_day=14):
    """
    Extract features from prediction ensemble for a given evaluation day.
    
    Args:
        predictions: N×H matrix of predictions (N runs × H horizons)
        current_price: Current market price
        eval_day: Which day ahead to evaluate (1-14)
    
    Returns:
        dict of features or None if invalid
    """
    if predictions is None or len(predictions) == 0:
        return None
    
    day_preds = predictions[:, eval_day - 1]
    
    return {
        'directional_consensus': np.mean(day_preds > current_price),
        'expected_return': (np.median(day_preds) - current_price) / current_price,
        'uncertainty': (np.percentile(day_preds, 75) - np.percentile(day_preds, 25)) / np.median(day_preds),
        'skewness': float(pd.Series(day_preds).skew()),
        'prediction_range': (np.max(day_preds) - np.min(day_preds)) / current_price,
        'downside_risk': (np.percentile(day_preds, 10) - current_price) / current_price
    }

# COMMAND ----------

# Loop through all commodities
for CURRENT_COMMODITY in COMMODITY_CONFIGS.keys():
    print("\n" + "=" * 80)
    print(f"FEATURE IMPORTANCE ANALYSIS: {CURRENT_COMMODITY.upper()}")
    print("=" * 80)
    
    # Discover all model versions for this commodity
    print(f"\nDiscovering model versions...")
    
    synthetic_versions = []
    try:
        DATA_PATHS = get_data_paths(CURRENT_COMMODITY)
        synthetic_df = spark.table(DATA_PATHS['predictions']).select("model_version").distinct()
        synthetic_versions = [row.model_version for row in synthetic_df.collect()]
    except:
        pass
    
    real_versions = []
    try:
        real_versions = get_model_versions(CURRENT_COMMODITY)
    except:
        pass
    
    all_model_versions = list(set(synthetic_versions + real_versions))
    
    if len(all_model_versions) == 0:
        print(f"⚠️  No model versions found for {CURRENT_COMMODITY}")
        continue
    
    print(f"✓ Found {len(all_model_versions)} model versions")
    
    # Loop through each model version
    for MODEL_VERSION in all_model_versions:
        print(f"\n{'-' * 80}")
        print(f"MODEL: {MODEL_VERSION}")
        print(f"{'-' * 80}")
        
        MODEL_DATA_PATHS = get_data_paths(CURRENT_COMMODITY, MODEL_VERSION)
        
        # ----------------------------------------------------------------------
        # Load Data
        # ----------------------------------------------------------------------
        print(f"\nLoading prepared data...")
        
        try:
            prices = spark.table(get_data_paths(CURRENT_COMMODITY)['prices_prepared']).toPandas()
            prices['date'] = pd.to_datetime(prices['date']).dt.normalize()
            prices = prices.reset_index(drop=True)
            
            # Load prediction matrices for this model version
            if MODEL_VERSION.startswith('synthetic_'):
                matrices_path = MODEL_DATA_PATHS['prediction_matrices']
            else:
                matrices_path = MODEL_DATA_PATHS['prediction_matrices_real']
            
            with open(matrices_path, 'rb') as f:
                prediction_matrices = pickle.load(f)
            
            # Normalize prediction matrix keys
            prediction_matrices = {
                pd.Timestamp(k).normalize(): v 
                for k, v in prediction_matrices.items()
            }
            
            print(f"✓ Loaded {len(prices)} days of prices")
            print(f"✓ Loaded {len(prediction_matrices)} prediction matrices")
        
        except Exception as e:
            print(f"⚠️  Could not load data: {e}")
            continue
        
        # ----------------------------------------------------------------------
        # Build Feature Dataset
        # ----------------------------------------------------------------------
        print(f"\nExtracting features...")
        
        eval_day = 14
        feature_data = []
        
        # Create date-to-index mapping
        date_to_idx = {date: idx for idx, date in enumerate(prices['date'])}
        
        # Iterate over prediction matrices
        for pred_date, predictions in prediction_matrices.items():
            if pred_date not in date_to_idx:
                continue
            
            current_idx = date_to_idx[pred_date]
            
            # Need enough future data
            if current_idx + eval_day >= len(prices):
                continue
            
            current_price = prices.loc[current_idx, 'price']
            future_price = prices.loc[current_idx + eval_day, 'price']
            
            features = extract_features(predictions, current_price, eval_day)
            if features is None:
                continue
            
            actual_return = (future_price - current_price) / current_price
            
            feature_data.append({
                'date': pred_date,
                'current_price': current_price,
                'actual_return': actual_return,
                **features
            })
        
        if len(feature_data) == 0:
            print("⚠️  No features extracted")
            continue
        
        feature_df = pd.DataFrame(feature_data)
        print(f"✓ Extracted features for {len(feature_df)} days")
        
        # ----------------------------------------------------------------------
        # Train Random Forest Model
        # ----------------------------------------------------------------------
        print(f"\nTraining Random Forest model...")
        
        feature_cols = ['directional_consensus', 'expected_return', 'uncertainty', 
                       'skewness', 'prediction_range', 'downside_risk']
        
        X = feature_df[feature_cols]
        y = feature_df['actual_return']
        
        rf_model = RandomForestRegressor(
            n_estimators=100,
            max_depth=10,
            random_state=42,
            n_jobs=-1
        )
        
        rf_model.fit(X, y)
        
        cv_scores = cross_val_score(rf_model, X, y, cv=5, scoring='r2')
        
        print(f"✓ Model trained")
        print(f"  R² score: {rf_model.score(X, y):.3f}")
        print(f"  CV R² score: {cv_scores.mean():.3f} (+/- {cv_scores.std():.3f})")
        
        # ----------------------------------------------------------------------
        # Feature Importance
        # ----------------------------------------------------------------------
        print(f"\nAnalyzing feature importance...")
        
        importances = rf_model.feature_importances_
        importance_df = pd.DataFrame({
            'feature': feature_cols,
            'importance': importances
        }).sort_values('importance', ascending=False)
        
        print("\nFeature Importances:")
        for _, row in importance_df.iterrows():
            print(f"  {row['feature']:25s}: {row['importance']:.3f}")
        
        # ----------------------------------------------------------------------
        # Visualization
        # ----------------------------------------------------------------------
        print(f"\nGenerating visualization...")
        
        fig, ax = plt.subplots(figsize=(10, 6))
        
        ax.barh(importance_df['feature'], importance_df['importance'], color='steelblue', alpha=0.7)
        ax.set_xlabel('Importance', fontsize=12)
        ax.set_title(f'Feature Importance - {CURRENT_COMMODITY.upper()} - {MODEL_VERSION}', 
                     fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='x')
        
        plt.tight_layout()
        
        viz_path = f'{VOLUME_PATH}/feature_importance_{CURRENT_COMMODITY}_{MODEL_VERSION}.png'
        plt.savefig(viz_path, dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {viz_path}")
        plt.show()
        plt.close()
        
        # ----------------------------------------------------------------------
        # Save Results
        # ----------------------------------------------------------------------
        print(f"\nSaving results...")
        
        feature_analysis = {
            'commodity': CURRENT_COMMODITY,
            'model_version': MODEL_VERSION,
            'feature_importance': importance_df,
            'model': rf_model,
            'feature_data': feature_df,
            'cv_scores': cv_scores,
            'r2_score': rf_model.score(X, y)
        }
        
        with open(MODEL_DATA_PATHS['feature_analysis'], 'wb') as f:
            pickle.dump(feature_analysis, f)
        
        print(f"✓ Saved: {MODEL_DATA_PATHS['feature_analysis']}")
        
        importance_csv = f'{VOLUME_PATH}/feature_importance_{CURRENT_COMMODITY}_{MODEL_VERSION}.csv'
        importance_df.to_csv(importance_csv, index=False)
        print(f"✓ Saved: {importance_csv}")
        
        print(f"\n✓ Feature analysis complete for {MODEL_VERSION}")
    
    print(f"\n{'=' * 80}")
    print(f"✓ {CURRENT_COMMODITY.upper()} COMPLETE")
    print(f"{'=' * 80}")

# COMMAND ----------

print("\n" + "=" * 80)
print("ALL FEATURE IMPORTANCE ANALYSES COMPLETE")
print("=" * 80)
print(f"Commodities analyzed: {', '.join([c.upper() for c in COMMODITY_CONFIGS.keys()])}")
print("\n✓ Feature importance analysis complete for all commodities and models")


FEATURE IMPORTANCE ANALYSIS: COFFEE

Discovering model versions...
✓ Found 16 model versions

--------------------------------------------------------------------------------
MODEL: random_walk_v1_test
--------------------------------------------------------------------------------

Loading prepared data...
✓ Loaded 965 days of prices
✓ Loaded 1 prediction matrices

Extracting features...
⚠️  No features extracted

--------------------------------------------------------------------------------
MODEL: arima_v1
--------------------------------------------------------------------------------

Loading prepared data...
✓ Loaded 965 days of prices
✓ Loaded 1 prediction matrices

Extracting features...
⚠️  No features extracted

--------------------------------------------------------------------------------
MODEL: sarimax_auto_weather_v1
--------------------------------------------------------------------------------

Loading prepared data...
✓ Loaded 965 days of prices
✓ Loaded 41 predict


Saving results...
✓ Saved: /Volumes/commodity/trading_agent/files/feature_analysis_coffee_synthetic_acc70.pkl
✓ Saved: /Volumes/commodity/trading_agent/files/feature_importance_coffee_synthetic_acc70.csv

✓ Feature analysis complete for synthetic_acc70

--------------------------------------------------------------------------------
MODEL: sarimax_weather_v1
--------------------------------------------------------------------------------

Loading prepared data...
✓ Loaded 965 days of prices
✓ Loaded 1 prediction matrices

Extracting features...
⚠️  No features extracted

--------------------------------------------------------------------------------
MODEL: prophet_v1
--------------------------------------------------------------------------------

Loading prepared data...
✓ Loaded 965 days of prices
✓ Loaded 42 prediction matrices

Extracting features...
⚠️  No features extracted

--------------------------------------------------------------------------------
MODEL: synthetic_acc80


COMMODITY CONFIGURATIONS

COFFEE:
  Harvest volume: 50 tons/year
  Harvest windows: [(5, 9)]
  Total harvest weeks: 21
  Weekly harvest rate: 2.38 tons/week

  Costs (percentage-based):
    Storage: 0.025% of value per day
    Transaction: 0.25% of sale value
    Max holding: 365 days from harvest start

  Example at $150/ton:
    Transaction cost (full harvest): $18.75
    Storage per day (full harvest): $1.88
    Storage per month (full harvest): $56.25
    Storage for 6 months: $337.50

Real Prediction Data:
  ✓ Real prediction data found in table: commodity.forecast.distributions
  Model versions available: 12
    - arima_111_v1
    - arima_v1
    - naive
    - naive_baseline
    - prophet_v1
    - random_walk_baseline
    - random_walk_v1
    - random_walk_v1_test
    - sarimax_auto_weather_v1
    - sarimax_weather_v1
    - xgboost
    - xgboost_weather_v1

SUGAR:
  Harvest volume: 50 tons/year
  Harvest windows: [(10, 12)]
  Total harvest weeks: 12
  Weekly harvest rate: 4.17 ton

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
# PROPER DIAGNOSTIC - Check overlap correctly

import pandas as pd
import pickle

TEST_COMMODITY = 'coffee'
TEST_MODEL = 'prophet_v1'

MODEL_DATA_PATHS = get_data_paths(TEST_COMMODITY, TEST_MODEL)

prices = spark.table(get_data_paths(TEST_COMMODITY)['prices_prepared']).toPandas()
prices['date'] = pd.to_datetime(prices['date']).dt.normalize()
prices = prices.reset_index(drop=True)

matrices_path = MODEL_DATA_PATHS['prediction_matrices_real']
with open(matrices_path, 'rb') as f:
    prediction_matrices = pickle.load(f)

prediction_matrices = {
    pd.Timestamp(k).normalize(): v 
    for k, v in prediction_matrices.items()
}

eval_day = 14
date_to_idx = {date: idx for idx, date in enumerate(prices['date'])}

print(f"Prices: {len(prices)} days ({prices['date'].min()} to {prices['date'].max()})")
print(f"Predictions: {len(prediction_matrices)}")

pred_dates = sorted(prediction_matrices.keys())
print(f"Prediction range: {pred_dates[0]} to {pred_dates[-1]}")

print(f"\nChecking all 42 predictions:")
not_in_prices = 0
too_late = 0
valid = 0

for pred_date in pred_dates:
    if pred_date not in date_to_idx:
        not_in_prices += 1
        continue
    
    idx = date_to_idx[pred_date]
    if idx + eval_day >= len(prices):
        too_late += 1
        print(f"  {pred_date}: idx={idx}, TOO LATE (idx+14={idx+14} >= {len(prices)})")
    else:
        valid += 1
        print(f"  {pred_date}: idx={idx}, VALID")

print(f"\nSummary:")
print(f"  Not in price data: {not_in_prices}")
print(f"  Too late to look ahead: {too_late}")
print(f"  Valid: {valid}")

Prices: 965 days (2022-01-03 00:00:00 to 2025-10-31 00:00:00)
Predictions: 42
Prediction range: 2018-07-06 00:00:00 to 2025-11-01 00:00:00

Checking all 42 predictions:

Summary:
  Not in price data: 42
  Too late to look ahead: 0
  Valid: 0
